<div class="alert alert-block alert-info">
<center><img src='webscraping.png' width='400'/></center>
<h1><center>WEB SCRAPING PYTHON 2.0 Hotels</center></h1>
    <hr>
    <h3><center>Description:</center></h3>
    <p>This is a second way to make a WS (WebScraping) on TripAdvisor web page to get hte comments. Actually, there are many ways to create a web crawler but to be practice, we will use Selenium to interact on the entire web page making clicks and moving through the links</p>
</div>

<div class="alert alert-block alert-warning">
<h3><center>Imports</center></h3>
    <hr>
    <p>Al the imports need to run this project</p>
</div>

In [1]:
###
# Libraries system or commonly called: Utils
###
import os
import time
import datetime
from datetime import date
import csv
import random
print("Libraries 'Utils' ready...")
###
# Selenium
###
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
    print("Selenium libraries ready...")
except ImportError as error:
    print("There is not a Module named 'Selenium' installed, please run the next cell...")
###
# urllib and BautifulSoup
###
try:
    import urllib
    from bs4 import BeautifulSoup
    print("BeautifulSoup and urllib libraries ready...")
except ImportError as error:
    print("There is not a Module named 'BeautifulSoup' installed, please run the next cell..")
###
# Forex -> Procing of the currencies
###
try:
    from forex_python.converter import CurrencyRates
    print("CurrencyRates libraries ready...")
except ImportError as error:
    print("There is not a Module named 'CurrencyRates' installed, please run the next cell...")
###
# Pandas
###
try:
    import pandas as pd
    print("Pandas libraries ready...")
except ImportError as error:
    print("There is not a Module named 'pandas' installed, please tun the next cell...")

Libraries 'Utils' ready...
Selenium libraries ready...
BeautifulSoup and urllib libraries ready...
CurrencyRates libraries ready...
Pandas libraries ready...


<div class="alert alert-warning" role="alert">
  <b>In case of Selenium is not installed</b>, run the next cell. Important!, althoug you had been installed Selenium that not mean that all Selenium package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install selenium

<div class="alert alert-warning" role="alert">
  <b>In case of BeautifulSoup is not installed</b>, run the next cell. Important!, althoug you had been installed BeautifulSoup that not mean that all BeautifulSoup package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install beautifulsoup4 

<div class="alert alert-warning" role="alert">
  <b>In case of forex-python is not installed</b>, run the next cell. Important!, althoug you had been installed forex-python that not mean that all forex-python package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install forex-python

<div class="alert alert-warning" role="alert">
  <b>In case of Pandas is not installed</b>, run the next cell. Important!, althoug you had been installed Pandas that not mean that all Pandas package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install pandas

<div class="alert alert-block alert-danger">
<h3><center>Importan!</center></h3>
    <hr>
    <p>Remember, <b>DON´T RUN</b> the pip install with -U beacuse of you will breake your Anaconda Enviroment due to uncompatible versions of the different libraries. Just RUN the pip install <b>WITHOUT -U</b> argument.</p>
</div>

<div class="alert alert-block alert-warning">
<h3><center>Methods</center></h3>
</div>

In [2]:
###
# Methods for handling SELENIUM
###
def connect(url):
    """This method will connect to the web site using selenium
       Parameters:
       ----------
           url: str, the url of the web page to scrap
       Returns:
       -------
           driver: ChromeDriver, the browser 
       Example:
       -------
           >>> connect('http://www.somepage.com')"""
    chromeOptions = webdriver.ChromeOptions()
    #chromeOptions.add_argument("--incognito")
    driver = webdriver.Chrome("chromedriver.exe",options=chromeOptions)
    driver.get(url) 
    wait(3)
    return driver

def quit(driver):
    """This method will diconect the selenium web driver and closing the chrome windows
       Parameters:
       Returns:
       Example:"""
    print("Fishing the selenium web driver")
    driver.quit()
    
def page_Ready(driver):
    """Method uses to wxecute some jquery code to wait until the page is ready
       Parameters:
       Returns:
       Example:"""
    page_state = driver.execute_script('return document.readyState;')
    return page_state
    
def is_Ready(driver):
    """The page ready method, make infinite loop to wait until web page has been downladed.
       Parameters:
       Returns:
       Example:"""
    while True:
        page_state = page_Ready(driver)
        if page_state == 'complete':
            print("Page ready...")
            break
            
def tab_manage(driver,num):
    """Method used to handdle the tabs, moving through them
       Parameters:
       Returns:
       Example:"""
    try:
        driver.switch_to.window(driver.window_handles[num])
        wait(3)
    except:
        print("Error handdling the tabs ")
        
def tab_closer(driver):
    """Method used to close the currently tab
       Parameters:
       Returns:
       Example:"""
    try:
        driver.close()
    except:
        print("\x1b[31m\"Error so bad closing the tab \"\x1b[0m")

In [3]:
###
# Methods to create waits
###
def explicit_wait(driver,many,key,path_element,num):
    """This method is create to create a explicit wait
       Parameters:
       Returns:
       Example:"""
    dicc_Bys={'selector':By.CSS_SELECTOR,'id':By.ID,
              'tag':By.TAG_NAME,'xpath':By.XPATH}
    try:
        if many:
            element = WebDriverWait(driver,num).until(
                EC.presence_of_all_elements_located(
                    (dicc_Bys[key],path_element)
                )
            )
        else:
            element = WebDriverWait(driver,num).until(
                EC.presence_of_element_located(
                    (dicc_Bys[key],path_element)
                )
            )
    except:
        print("\x1b[31m\"Something bad happen with the explicit wait \"\x1b[0m")
    return element

def implicit_wait(driver,many,key,path_element,num):
    """Method used to handle a implicit wait:
       Parameters:
       Returns:
       Example:"""
    driver.implicitly_wait(num)
    try:
        if many:
            if key == 'id':
                element = driver.find_element_by_id(path_element)
            if key == 'selector':
                element = driver.find_elements_by_css_selector(path_element)
            if key ==  'xpath':
                element = driver.find_elements_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_elements_by_tag_name(path_element)
        else:
            if key == 'selector':
                element = driver.find_element_by_css_selector(path_element)
            if key == 'xpath':
                element = driver.find_element_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_element_by_tag_name(path_element)
    except:
        print("\x1b[31m\"Something bad happen with the implicit wait function \"\x1b[0m")
    return element

def hard_wait(driver,many,key,path_element,num):
    """This method create a system wait fro handdle in genereal all the Ajax interactions
       Parameters:
       Returns:
       Example:"""
    time.sleep(num)
    try:
        if many:
            if key == 'id':
                element = driver.find_element_by_id(path_element)
            if key == 'selector':
                element = driver.find_elements_by_css_selector(path_element)
            if key ==  'xpath':
                element = driver.find_elements_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_elements_by_tag_name(path_element)
        else:
            if key == 'selector':
                element = driver.find_element_by_css_selector(path_element)
            if key == 'xpath':
                element = driver.find_element_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_element_by_tag_name(path_element)
    except:
        print("\x1b[31m\"Something bad happen with the hard wait function \"\x1b[0m")
    return element

def wait(num):
    """This method create a system wait fro handdle in genereal all the Ajax interactions
       Parameters:
       Returns:
       Example:"""
    time.sleep(num)
    return None

def waits_and_elements(driver,many,key,path_element,time,nombre):
    """Method used to handdle all waits
       Parameters:
       Returns:
       Example:"""
    try:
        elements = explicit_wait(driver,many,key,path_element,time)
    except:
        try:
            elements = implicit_wait(driver,many,key,path_element,time)
            if elements == []:
                elements = hard_wait(driver,many,key,path_element,time)
        except:
            print("\x1b[31m\"Something really bad happend, with: "+nombre+" "+path_element+", could be a change of the HTML DOM \"\x1b[0m")
    return elements

In [4]:
###
# Handdling buttons
### 
def random_translator(driver,segs):
    """Method used to handdle the button of translate a comment.
       Parameters:
       Returns:
       Example:"""
    str_div = 'div[class="ui_radio"]'
    elements = waits_and_elements(driver,True,'selector',str_div,segs,'botón_para_traducir')
    try:
        elements[-1].click()
    except:
        print("\x1b[31m\"Random translator is not here\"\x1b[0m")
        
def next_page(driver,segs):
    """Mehtod used to navigate across the next pages buttons
       Parameters:
       Returns:
       Example:"""
    str_next_b = 'a[class="ui_button nav next primary "]'
    try:
        element = waits_and_elements(driver,False,'selector',str_next_b,segs,'botón_para_cambiar_página')
        element.click()
        is_Ready(driver)
        return True
    except:
        print("\x1b[31m\"There is no more pages to make click\"\x1b[0m")
        return False
        
def is_Lenguage(driver,segs):
    """Metod used to make click over the idiom botton
       Parameters:
       Returns:
       Example:"""
    try:
        str_button = 'li[class="ui_radio location-review-review-list-parts-ReviewFilter__filter_row--p0z3u location-review-review-list-parts-ReviewFilter__cx_brand_refresh_phase2--2riTx"]'
        objects = waits_and_elements(driver,True,'selector',str_button,segs,'botones_radio_idioma')
        diccionario = {
            ('español' if 'español' in object_.text else 'inglés' if 'inglés' in object_.text else 'ignore'):object_ for object_ in objects 
        }   
    except:
        print("\x1b[31m\"Lenguages was not obtaned\"\x1b[0m")
    return diccionario

####I DO NOT KNOW WHY, BUT SHOW MORE ONLY WORKS AFTER YOU INTERACT WITH THE PAGE (MAKING CLICKS ETC ETC).
def show_more(driver):
    """Method use to make click over the comment area to show the full comment
       Parameters:
       Returns:
       Example:"""
    str_show_more = 'div[class="location-review-review-list-parts-ExpandableReview__ctaLine--24Qlb"]'
    elements = waits_and_elements(driver,False,'selector',str_show_more,10,'botón_ver_más')
    try:
        element.click()
    except:
        print("\x1b[31m\"Show more is not here...\"\x1b[0m")

In [5]:
###
# Methods to extract the general information about the hotel
###
def header_information(driver,segs):
    """This method will interact only with thw header of the page, just to get the information about he hotel
       Parameters:
       Returns:
       Example:"""
    str_header = 'atf_header_wrap'
    try:
        element = waits_and_elements(driver,False,'id',str_header,segs,'contenedor_header')
        try: # Name
            name = waits_and_elements(element,False,'id','HEADING',segs,'nombre_del_hotel').text
        except:
            name = "Sin_nombre"
        try: # Direction
            str_span = 'span[class="public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI"]'
            dirc = waits_and_elements(element,False,'selector',str_span,segs,'dirección_hotel').text
        except:
            dirc = "Sin_dirección"
        try: # Score
            str_scr = 'span[class="hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA"]'
            scrt = waits_and_elements(driver,False,'selector',str_scr,segs,'califiacion_del_hotel').text
        except:
            scrt = "Sin_score"
        try: # Phone
            str_phon = 'span[class="public-business-listing-ContactInfo__nonWebLinkText--nGymU public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI"]'
            phne = waits_and_elements(element,False,'selector',str_phon,segs,'telefono_hotel').text
        except:
            phne = "Sin_teléfono"
        try:
            str_site = 'div[class="hotels-hotel-review-atf-info-parts-BusinessListingEntry__entry--210S0"]'
            wait(1)
            waits_and_elements(element,False,'selector',str_site,segs,'sitio_web_hotel').click()
            tab_manage(driver,-1)
            site = driver.current_url
            tab_closer(driver)
            tab_manage(driver,0)
        except:
            site = "Sin_sitio"
    except:
        print("\x1b[31m\"Exception: HEADER.Something bad happed with header_information, the element could no be located\"\x1b[0m")
    return name,dirc,scrt,phne,site

In [6]:
###
# Attributes Score
###
def clean_second_dicc(dicc):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    list_attributes = ["Calidad/precio",
                       "Habitaciones",
                       "Ubicación",
                       "Limpieza",
                       "Servicio",
                       "Calidad del descanso"]
    for element in list_attributes:
        if element not in dicc.keys():
            dicc[element] = 'No_Calificado'
    return dicc

In [7]:
###
# Methods to extract all the information about the comments
###
def all_comment(driver,segs):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    str_div = 'div[class="hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H"]'
    list_usuario = []
    list_perfil = []
    list_localidad = []
    list_fecha_comentario = []
    list_score = []
    list_titulo = []
    list_coment = []
    list_fecha_visita = []
    list_tipo_viajero = []
    list_respuesta = []
    list_diccss = []
    try:
        elements = waits_and_elements(driver,True,'selector',str_div,segs,'div_comentarios')
        for element in elements:
            try: # User and Profile
                str_a = 'a[class="ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC"]'
                object_ = waits_and_elements(element,False,'selector',str_a,segs,'usuario_y_perfil')
                user = object_.text
                profile = object_.get_attribute('href')
            except:
                user = "Sin_usuario"
                profile = "Sin_perfil"
            try: # Ubication of the user
                str_ubc = 'span[class="default social-member-common-MemberHometown__hometown--3kM9S small"]'
                ubication = waits_and_elements(element,False,'selector',str_ubc,segs,'ubicación_usuario').text
            except:
                ubication = "Sin_ubicacion"
            try: # Date of comment
                str_fecha_coment = 'div[class="social-member-event-MemberEventOnObjectBlock__event_type--3njyv"]'
                date = waits_and_elements(element,False,'selector',str_fecha_coment,segs,'fecha_del_comentario')
                text = waits_and_elements(date,False,'tag','span',segs,'fecha_del_comentario').text
                fecha = text.split('(')[1].replace(")","")
            except:
                fecha = "Sin_fecha_del_comentario"
            try: # Score
                str_div = 'div[class="location-review-review-list-parts-RatingLine__bubbles--GcJvM"]'
                abc = waits_and_elements(element,False,'selector',str_div,segs,'Califiación_usuario_hotel')
                score = waits_and_elements(abc,False,'tag','span',segs,'califiación_usuario_hotel').get_attribute('class')
                score = score.replace('ui_bubble_rating bubble_','')
            except:
                score = "Sin_calificacion"
            try: # Header comment
                str_coment_titulo = 'a[class="location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT"]'
                header = waits_and_elements(element,False,'selector',str_coment_titulo,segs,'comentario_título').text
            except:
                header = "Sin_titulo"
            try: #the show more
                str_show_more = 'div[class="location-review-review-list-parts-ExpandableReview__ctaLine--24Qlb"]'
                waits_and_elements(element,False,'selector',str_show_more,segs,'botón_ver_más').click()
            except:
                print("\x1b[31m\"Clicking again show more \"\x1b[0m")
                str_show_more = 'div[class="location-review-review-list-parts-ExpandableReview__ctaLine--24Qlb"]'
                waits_and_elements(element,False,'selector',str_show_more,segs,'botón_ver_más')
            try: # Comment apply 
                #show_more(element)
                str_coment_body = 'q[class="location-review-review-list-parts-ExpandableReview__reviewText--gOmRC"]'
                comment = waits_and_elements(element,False,'selector',str_coment_body,segs,'cuerpo_comentario').text
                if '\n' in comment:
                    comment = comment.replace("\n"," ")
            except:
                comment = "Sin_comentario"
            try: # Date of visit
                str_date_visit = 'span[class="location-review-review-list-parts-EventDate__event_date--1epHa"]'
                date_of_visit = waits_and_elements(element,False,'selector',str_date_visit,segs,'fecha_visita').text.replace('Fecha de la estadía: ','')
            except:
                date_of_visit = "Sin_fecha_de_visita"
            try: # Traveler type
                str_client = 'span[class="location-review-review-list-parts-TripType__trip_type--3w17i"]'
                client_type = waits_and_elements(element,False,'selector',str_client,segs,'tipo_viajero').text.replace('Tipo de viaje: ','')
            except:
                client_type = "Sin_tipo_de_viajero"
            try: # Response
                str_respuesta = 'span[class="location-review-review-list-parts-OwnerResponse__reviewText--108-V"]'
                response = waits_and_elements(element,False,'selector',str_respuesta,segs,'respuesta').text
                if '\n' in response:
                    response = response.replace("\n"," ")
            except:
                response = "Sin_respuesta"
            try: 
                str_div = 'div[class="location-review-review-list-parts-AdditionalRatings__rating--1_G5W location-review-review-list-parts-AdditionalRatings__large--IOg2u"]'
                str_cal = 'span[class*="ui_bubble_rating"]'
                divs = waits_and_elements(element,True,'selector',str_div,segs,'divs_evaluaciones')
                dicc = {}
                for item in divs:
                    bubble =  waits_and_elements(item,False,'selector',str_cal,segs,'bubble').get_attribute('class')
                    bubble = bubble[-2:]
                    print(bubble)
                    dicc[item.text] = bubble
                clean_second_dicc(dicc)
                print(dicc)
            except:
                dicc = {'Calidad/precio': 'No_Calificado', 
                        'Habitaciones': 'No_Calificado', 
                        'Ubicación': 'No_Calificado', 
                        'Limpieza': 'No_Calificado', 
                        'Servicio': 'No_Calificado', 
                        'Calidad del descanso': 'No_Calificado'}
            list_usuario.append(user)
            list_perfil.append(profile)
            list_localidad.append(ubication)
            list_fecha_comentario.append(fecha)
            list_score.append(score)
            list_titulo.append(header)
            list_coment.append(comment)
            list_fecha_visita.append(date_of_visit)
            list_tipo_viajero.append(client_type)
            list_respuesta.append(response)
            list_diccss.append(dicc)
    except:
        print("\x1b[31m\"Exception:BODY_COMMENTS Something bad happend with all_comment, the element could ni be located\"\x1b[0m")
    return list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo_viajero,list_respuesta,list_diccss

In [8]:
###
# Date of consult, filters diccs and save file
###
def date_consult(is_finally=True):
    """Método utilizado para crear la fecha de consulta
       Parámetros: <None>
       Return: <date> tipo: date --> Fecha de salida del sistema"""
    if is_finally:
        now = datetime.datetime.now()
        now = now.strftime('%Y-%m-%d_%H-%M-%S')
    else:
        now = date.today()
        now = str(now)
    return now

def filter_dicc(dicc_mask,dicc_real):#######################################
    """Método utilizado para eliminar los filtros que el usuario no desee al hacer web scrapin
       Párametros: <dicc_mask> tipo: dicc --> Diccionario de true o false que sera un hiperparametro del usuario
                   <dicc_real> tipo: dicc --> Diccionario que contiene los botones para los filtros de selenium
       Retorno: <dicc_real> tipo dicc --> Diccionario ya filtrado de los elementos de selenium"""
    for key,val in dicc_mask.items():
        if val == False:
            dicc_real.pop(key)
    return dicc_real

def save(df,name,date,filter_=False,describe=False,mode_append=True):
    """Método para guardar el df resultante
       Parámetros: <df>   tipo: Pandas.DataFrame --> El dataframe con la información
                   <name> tipo: str --> El nombre del establesimiento
                   <date> tipo: datime.date --> La fecha en la que se realizo la consulta
       Retorno: <None>"""
    if describe:
        df[['Filtro_1','Filtro_2','Filtro_3']] = df.Categoria.str.split("_",expand=True) 
    if " " in name: name = name.replace(" ","_")
    if filter_: name = name+"_"+"filtered"
    if not mode_append:
        df.to_csv(name+"_"+date+'_pipes.csv',sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
        df.to_csv(name+"_"+date+'_comma.csv',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
        print("Datos correctamente guardados")
    else:
        df.to_csv(name+"_"+date+'_pipes.csv',sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig",mode='a')
        df.to_csv(name+"_"+date+'_comma.csv',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig",mode='a')
    return None

In [9]:
###
# Method to run all over the page
### 
def run_all(url,name_file,esp=True,eng=True,max_page=None,segs=0.5):
    if max_page == None: iterator = 10000
    else: iterator = max_page
    dicc_data = {
        'Establecimiento':[],
        'Direccion':[],
        'Calificacion':[],
        'Telefono':[],
        'Sitio':[],
        'Fecha_consulta':[],
        "Fecha_comentario":[],
        "Fecha_estancia":[],
        "Usuario":[],
        "Localidad":[],
        "Perfil":[],
        "Comentario_Titulo":[],
        "Comentario":[],
        "Respuesta":[],
        "Idioma":[],
        "C_comentario":[],
        "Tipo_cliente":[],
        "Calidad/precio":[],
        "Habitaciones":[],
        "Ubicación":[],
        "Limpieza":[],
        "Servicio":[],
        "Calidad del descanso":[],
        "Pagina":[]
    }
    df = pd.DataFrame(dicc_data)
    driver = connect(url)
    is_Ready(driver)
    name,dirc,scrt,phne,site = header_information(driver,segs)
    print(name,dirc,scrt,phne,site)
    dicc_idm_m = {'español':esp,
                  'inglés':eng}
    _idiom = is_Lenguage(driver,segs)
    _idiom = filter_dicc(dicc_idm_m,_idiom)
    print(_idiom)
    _idiom.pop('ignore')
    str_actual_page = 'span[class="pageNum current cx_brand_refresh_phase2 disabled"]'
    ###Flata copiar lo del proceso anterior
    for k_idiom,v_idiom in _idiom.items():
        try: v_idiom.click()
        except StaleElementReferenceException as e: wait(2); driver.execute_script('arguments[0].click()', v_idiom)
        except: print("ERROR MAKING CLICK...")
        is_Ready(driver)
        aux = iterator #10000
        while aux > 0:
            wait(1)
            try: pagina = driver.find_element_by_css_selector(str_actual_page).text; print("IDIOMA: ",k_idiom,"PAGINA: ",driver.find_element_by_css_selector(str_actual_page).text)
            except: print("PAGINA SIN MÁS ELEMENTOS POR ITERAR")
            #random_translator(driver,segs)
            list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo_viajero,list_respuesta,list_diccss = all_comment(driver,segs)
            print(list_usuario)
            print(list_coment)
            for indx in range(len(list_coment)):
                dicc_data = {
                    'Establecimiento':name,
                    'Direccion':dirc,
                    'Calificacion':scrt,
                    'Telefono':phne,
                    'Sitio':site,
                    'Fecha_consulta':date_consult(is_finally=False),
                    "Fecha_comentario":list_fecha_comentario[indx],
                    "Fecha_estancia":list_fecha_visita[indx],
                    "Usuario":list_usuario[indx], 
                    "Localidad":list_localidad[indx],
                    "Perfil":list_perfil[indx],
                    "Comentario_Titulo":list_titulo[indx],
                    "Comentario":list_coment[indx],
                    "Respuesta":list_respuesta[indx],
                    "Idioma":k_idiom,
                    "C_comentario":list_score[indx],
                    "Tipo_cliente":list_tipo_viajero[indx],
                    "Calidad/precio":list_diccss[indx]["Calidad/precio"],
                    "Habitaciones":list_diccss[indx]["Habitaciones"],
                    "Ubicación":list_diccss[indx]["Ubicación"],
                    "Limpieza":list_diccss[indx]["Limpieza"],
                    "Servicio":list_diccss[indx]["Servicio"],
                    "Calidad del descanso":list_diccss[indx]["Calidad del descanso"],
                    "Pagina":pagina
                }
                df = df.append(dicc_data,ignore_index=True)
            save(df,name_file,date_consult(is_finally=False),mode_append=True)
            aux -= 1
            if next_page(driver,segs) == False:
                break
    df.drop_duplicates(['Usuario','Comentario'],keep='last',inplace=True)
    save(df,name_file,date_consult(is_finally=True),mode_append=False)
    quit(driver)

In [10]:
###
# Method to run all over the page
### 
def run_all_es(url,name_file,max_page=None,segs=0.5):
    if max_page == None: iterator = 10000
    else: iterator = max_page
    dicc_data = {
        'Establecimiento':[],
        'Direccion':[],
        'Calificacion':[],
        'Telefono':[],
        'Sitio':[],
        'Fecha_consulta':[],
        "Fecha_comentario":[],
        "Fecha_estancia":[],
        "Usuario":[],
        "Localidad":[],
        "Perfil":[],
        "Comentario_Titulo":[],
        "Comentario":[],
        "Respuesta":[],
        "Idioma":[],
        "C_comentario":[],
        "Tipo_cliente":[],
        "Calidad/precio":[],
        "Habitaciones":[],
        "Ubicación":[],
        "Limpieza":[],
        "Servicio":[],
        "Calidad del descanso":[],
        "Pagina":[]
    }
    df = pd.DataFrame(dicc_data)
    driver = connect(url)
    is_Ready(driver)
    name,dirc,scrt,phne,site = header_information(driver,segs)
    print(name,dirc,scrt,phne,site)
    #dicc_idm_m = {'español':esp,
    #              'inglés':eng}
    _idiom = is_Lenguage(driver,segs)
    #_idiom = filter_dicc(dicc_idm_m,_idiom)
    print(_idiom)
    _idiom.pop('ignore')
    _idiom.pop('inglés')
    str_actual_page = 'span[class="pageNum current cx_brand_refresh_phase2 disabled"]'
    ###Flata copiar lo del proceso anterior
    for k_idiom,v_idiom in _idiom.items():
        try: v_idiom.click()
        except StaleElementReferenceException as e: wait(2); driver.execute_script('arguments[0].click()', v_idiom)
        except: print("ERROR MAKING CLICK...")
        is_Ready(driver)
        aux = iterator #10000
        while aux > 0:
            wait(1)
            try: pagina = driver.find_element_by_css_selector(str_actual_page).text; print("IDIOMA: ",k_idiom,"PAGINA: ",driver.find_element_by_css_selector(str_actual_page).text)
            except: print("PAGINA SIN MÁS ELEMENTOS POR ITERAR")
            #random_translator(driver,segs)
            list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo_viajero,list_respuesta,list_diccss = all_comment(driver,segs)
            print(list_usuario)
            print(list_coment)
            for indx in range(len(list_coment)):
                dicc_data = {
                    'Establecimiento':name,
                    'Direccion':dirc,
                    'Calificacion':scrt,
                    'Telefono':phne,
                    'Sitio':site,
                    'Fecha_consulta':date_consult(is_finally=False),
                    "Fecha_comentario":list_fecha_comentario[indx],
                    "Fecha_estancia":list_fecha_visita[indx],
                    "Usuario":list_usuario[indx], 
                    "Localidad":list_localidad[indx],
                    "Perfil":list_perfil[indx],
                    "Comentario_Titulo":list_titulo[indx],
                    "Comentario":list_coment[indx],
                    "Respuesta":list_respuesta[indx],
                    "Idioma":k_idiom,
                    "C_comentario":list_score[indx],
                    "Tipo_cliente":list_tipo_viajero[indx],
                    "Calidad/precio":list_diccss[indx]["Calidad/precio"],
                    "Habitaciones":list_diccss[indx]["Habitaciones"],
                    "Ubicación":list_diccss[indx]["Ubicación"],
                    "Limpieza":list_diccss[indx]["Limpieza"],
                    "Servicio":list_diccss[indx]["Servicio"],
                    "Calidad del descanso":list_diccss[indx]["Calidad del descanso"],
                    "Pagina":pagina
                }
                df = df.append(dicc_data,ignore_index=True)
            save(df,name_file,date_consult(is_finally=False),mode_append=True)
            aux -= 1
            if next_page(driver,segs) == False:
                break
    df.drop_duplicates(['Usuario','Comentario'],keep='last',inplace=True)
    save(df,name_file,date_consult(is_finally=True),mode_append=False)
    quit(driver)

In [11]:
###
# Method to run all over the page
### 
def run_all_en(url,name_file,max_page=None,segs=0.5):
    if max_page == None: iterator = 10000
    else: iterator = max_page
    dicc_data = {
        'Establecimiento':[],
        'Direccion':[],
        'Calificacion':[],
        'Telefono':[],
        'Sitio':[],
        'Fecha_consulta':[],
        "Fecha_comentario":[],
        "Fecha_estancia":[],
        "Usuario":[],
        "Localidad":[],
        "Perfil":[],
        "Comentario_Titulo":[],
        "Comentario":[],
        "Respuesta":[],
        "Idioma":[],
        "C_comentario":[],
        "Tipo_cliente":[],
        "Calidad/precio":[],
        "Habitaciones":[],
        "Ubicación":[],
        "Limpieza":[],
        "Servicio":[],
        "Calidad del descanso":[],
        "Pagina":[]
    }
    df = pd.DataFrame(dicc_data)
    driver = connect(url)
    is_Ready(driver)
    name,dirc,scrt,phne,site = header_information(driver,segs)
    print(name,dirc,scrt,phne,site)
    #dicc_idm_m = {'español':esp,
    #              'inglés':eng}
    _idiom = is_Lenguage(driver,segs)
    #_idiom = filter_dicc(dicc_idm_m,_idiom)
    print(_idiom)
    _idiom.pop('ignore')
    _idiom.pop('español')
    str_actual_page = 'span[class="pageNum current cx_brand_refresh_phase2 disabled"]'
    ###Flata copiar lo del proceso anterior
    for k_idiom,v_idiom in _idiom.items():
        try: v_idiom.click()
        except StaleElementReferenceException as e: wait(2); driver.execute_script('arguments[0].click()', v_idiom)
        except: print("ERROR MAKING CLICK...")
        is_Ready(driver)
        aux = iterator #10000
        while aux > 0:
            wait(1)
            try: pagina = driver.find_element_by_css_selector(str_actual_page).text; print("IDIOMA: ",k_idiom,"PAGINA: ",driver.find_element_by_css_selector(str_actual_page).text)
            except: print("PAGINA SIN MÁS ELEMENTOS POR ITERAR")
            #random_translator(driver,segs)
            list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo_viajero,list_respuesta,list_diccss = all_comment(driver,segs)
            print(list_usuario)
            print(list_coment)
            for indx in range(len(list_coment)):
                dicc_data = {
                    'Establecimiento':name,
                    'Direccion':dirc,
                    'Calificacion':scrt,
                    'Telefono':phne,
                    'Sitio':site,
                    'Fecha_consulta':date_consult(is_finally=False),
                    "Fecha_comentario":list_fecha_comentario[indx],
                    "Fecha_estancia":list_fecha_visita[indx],
                    "Usuario":list_usuario[indx], 
                    "Localidad":list_localidad[indx],
                    "Perfil":list_perfil[indx],
                    "Comentario_Titulo":list_titulo[indx],
                    "Comentario":list_coment[indx],
                    "Respuesta":list_respuesta[indx],
                    "Idioma":k_idiom,
                    "C_comentario":list_score[indx],
                    "Tipo_cliente":list_tipo_viajero[indx],
                    "Calidad/precio":list_diccss[indx]["Calidad/precio"],
                    "Habitaciones":list_diccss[indx]["Habitaciones"],
                    "Ubicación":list_diccss[indx]["Ubicación"],
                    "Limpieza":list_diccss[indx]["Limpieza"],
                    "Servicio":list_diccss[indx]["Servicio"],
                    "Calidad del descanso":list_diccss[indx]["Calidad del descanso"],
                    "Pagina":pagina
                }
                df = df.append(dicc_data,ignore_index=True)
            save(df,name_file,date_consult(is_finally=False),mode_append=True)
            aux -= 1
            if next_page(driver,segs) == False:
                break
    df.drop_duplicates(['Usuario','Comentario'],keep='last',inplace=True)
    save(df,name_file,date_consult(is_finally=True),mode_append=False)
    quit(driver)

In [12]:
###
# Get descriptivos
###
def _make_click(k,v):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    try:
        v.click()
    except StaleElementReferenceException as e: 
        wait(2)
        driver.execute_script('arguments[0].click()', v)
    except:
        print("Error making click over: ",k)
    wait(2)
        
def _a_button(driver,segs,dicc,*argv):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    str_a = 'a[class="pageNum cx_brand_refresh_phase2 "]'
    as_ = waits_and_elements(driver,True,'selector',str_a,segs,'As')
    if as_ != []:
        num = int(as_[-1].text)*5
        _make_click('a_last',as_[-1])
    else:
        num = 0
    str_divs = 'div[class="hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H"]'
    divs_ = waits_and_elements(driver,True,'selector',str_divs,segs,'DIVs')
    if divs_ != []:
        num = num + len(divs_)
    else:
        num = 0
    dicc['Categoria'].append(argv[0]+'_'+argv[1]+'_'+argv[2])
    dicc['Contador_Comentarios'].append(num)
    return dicc

def clean_big_dicc(dicc_):
    """Method used to clean the diccs
       Parameters:
       Returns:
       Example:"""
    dicc_scr = {k.split(' ')[0]:v for (k,v) in dicc_.items() if " " in k}
    dicc_prd = {k:v for (k,v) in dicc_.items() if "-" in k}
    dicc_tpo = {k:v for (k,v) in dicc_.items() if not("-" in k) and not(" " in k)}
    return dicc_scr,dicc_prd,dicc_tpo
    
def run_descriptivos(url,name,segs=0.5):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    driver = connect(url)
    is_Ready(driver)
    try:
        str_li = 'li[class="ui_checkbox location-review-review-list-parts-ReviewFilter__filter_row--p0z3u location-review-review-list-parts-ReviewFilter__cx_brand_refresh_phase2--2riTx"]'
        lis = waits_and_elements(driver,True,'selector',str_li,segs,'LIs')
        dicc = {"Categoria":[],
            "Contador_Comentarios":[]}
        df = pd.DataFrame(dicc)
        dicc_ = {}
        for element in lis:
            dicc_[element.text] = element
                 #Period
        dicc_idm = is_Lenguage(driver,segs)
        dicc_idm_m = {'español':True,
                  'inglés':True}
        dicc_idm = filter_dicc(dicc_idm_m,dicc_idm)
        dicc_idm.pop('ignore')
        dicc_scr,_,dicc_tpo = clean_big_dicc(dicc_)
        print(dicc_idm,dicc_scr,dicc_tpo)
        for k_idm,v_idm in dicc_idm.items():
            print(k_idm)
            _make_click(k_idm,v_idm)
            is_Ready(driver)
            dicc = _a_button(driver,segs,dicc,k_idm,"","")
            for k_scr,v_scr in dicc_scr.items():
                print(k_scr)
                _make_click(k_scr,v_scr)
                is_Ready(driver)
                dicc = _a_button(driver,segs,dicc,k_idm,k_scr,"")
                for k_tpo,v_tpo in dicc_tpo.items():
                    print(k_tpo)
                    _make_click(k_tpo,v_tpo)
                    is_Ready(driver)
                    dicc = _a_button(driver,segs,dicc,k_idm,k_scr,k_tpo)
                    _make_click(k_tpo,v_tpo)
                _make_click(k_scr,v_scr)   
    except:
        print("\x1b[31m\"DESCRIPTIVE: DESCRIPTIVE.Something bad happend getting the elements LI\"\x1b[0m")
    df = pd.DataFrame(dicc)
    save(df,name,date_consult(is_finally=True),mode_append=False)

<div class="alert alert-block alert-info">
<h3><center>Descriptivo</center></h3>
</div>

In [44]:
driver = connect('https://www.tripadvisor.com.mx/Hotel_Review-g150807-d152681-Reviews-Grand_Fiesta_Americana_Coral_Beach_Cancun-Cancun_Yucatan_Peninsula.html')

In [13]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): ")
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_descriptivos(url,name=name,segs=wait_)

Ingresa la url del sitio al que vamos a iterar: https://www.tripadvisor.com.mx/Hotel_Review-g150807-d152681-Reviews-Grand_Fiesta_Americana_Coral_Beach_Cancun-Cancun_Yucatan_Peninsula.html
Ingresa el nombre de tu archivo: FABC_DESC
Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): .5
Page ready...
{'inglés': <selenium.webdriver.remote.webelement.WebElement (session="f6a60941eac190c39bc768119d1f9c18", element="4327f2d8-474f-4a35-bd34-2b42c9f82e74")>, 'español': <selenium.webdriver.remote.webelement.WebElement (session="f6a60941eac190c39bc768119d1f9c18", element="5fe5d3b3-2c0c-4bc3-a60f-2125fff497fc")>} {'Excelente': <selenium.webdriver.remote.webelement.WebElement (session="f6a60941eac190c39bc768119d1f9c18", element="de4ee77f-a8b4-4854-8586-1ecbcd6380c1")>, 'Muy': <selenium.webdriver.remote.webelement.WebElement (session="f6a60941eac190c39bc768119d1f9c18", element="86093220-3d70-45e8-b0b4-ec52d519bd94")>, 'Regular': <selenium.webdriver.remote.webelement.WebE

<div class="alert alert-block alert-info">
<h3><center>Idioma español u inglés o ambos</center></h3>
</div>

In [ ]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = int(input("Ingresa el número máximo de páginas a iterar: "))
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_all(url,name,esp=False,eng=True,max_page=max_page,segs=wait_)

<div class="alert alert-block alert-info">
<h3><center>Idioma solo español</center></h3>
</div>

In [48]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = int(input("Ingresa el número máximo de páginas a iterar: "))
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_all_es(url,name,max_page=max_page,segs=wait_)

Ingresa la url del sitio al que vamos a iterar: https://www.tripadvisor.com.mx/Hotel_Review-g150807-d506374-Reviews-Live_Aqua_Beach_Resort_Cancun-Cancun_Yucatan_Peninsula.html
Ingresa el nombre de tu archivo: lat20_es
Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): .5
Ingresa el número máximo de páginas a iterar: 3
Page ready...
Live Aqua Beach Resort Cancún Boulevard Kukulcan, KM 12.5 Zona Hotelera, Cancún 77500 México 4.5 011 52 443 690 3093 https://reservations.fiestamericana.com/97941?utm_source=tripadvisor&utm_medium=display&utm_content=anunciopatrocinado&utm_campaign=1905-focosrojos
{'ignore': <selenium.webdriver.remote.webelement.WebElement (session="ee163a6d39d10312a006fc20d7d23179", element="0a3bc42d-fa27-4201-82fb-5792fa146973")>, 'inglés': <selenium.webdriver.remote.webelement.WebElement (session="ee163a6d39d10312a006fc20d7d23179", element="d165dc0b-23a7-4fa9-8d70-7dc6fde4a45d")>, 'español': <selenium.webdriver.remote.webelement.WebElement (se

"Something bad happen with the explicit wait "
"Something really bad happend, with: ubicación_usuario span[class="default social-member-common-MemberHometown__hometown--3kM9S small"], could be a change of the HTML DOM "
"Clicking again show more "
"Something bad happen with the explicit wait "
"Something really bad happend, with: divs_evaluaciones div[class="location-review-review-list-parts-AdditionalRatings__rating--1_G5W location-review-review-list-parts-AdditionalRatings__large--IOg2u"], could be a change of the HTML DOM "
"Clicking again show more "
"Something bad happen with the explicit wait "
"Something really bad happend, with: tipo_viajero span[class="location-review-review-list-parts-TripType__trip_type--3w17i"], could be a change of the HTML DOM "
"Something bad happen with the explicit wait "
"Something really bad happend, with: divs_evaluaciones div[class="location-review-review-list-parts-AdditionalRatings__rating--1_G5W location-review-review-list-parts-AdditionalRating

<div class="alert alert-block alert-info">
<h3><center>Idioma solo inglés</center></h3>
</div>

In [34]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = int(input("Ingresa el número máximo de páginas a iterar: "))
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 10
run_all_en(url,name,max_page=max_page,segs=wait)

Ingresa la url del sitio al que vamos a iterar: https://www.tripadvisor.com.mx/Hotel_Review-g150807-d506374-Reviews-Live_Aqua_Beach_Resort_Cancun-Cancun_Yucatan_Peninsula.html
Ingresa el nombre de tu archivo: lat20_en
Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): .5
Ingresa el número máximo de páginas a iterar: 2
Page ready...
Live Aqua Beach Resort Cancún Boulevard Kukulcan, KM 12.5 Zona Hotelera, Cancún 77500 México 4.5 011 52 443 690 3093 https://reservations.fiestamericana.com/97941?utm_source=tripadvisor&utm_medium=display&utm_content=anunciopatrocinado&utm_campaign=1905-focosrojos
{'ignore': <selenium.webdriver.remote.webelement.WebElement (session="1e6090630945ed6aa1ecfe2dd648c8c0", element="081e28d1-4778-4372-b6c1-43d6105f40fd")>, 'inglés': <selenium.webdriver.remote.webelement.WebElement (session="1e6090630945ed6aa1ecfe2dd648c8c0", element="89a6c72b-1da7-42fb-81b1-6033f452c67b")>, 'español': <selenium.webdriver.remote.webelement.WebElement (se

Page ready...
IDIOMA:  inglés PAGINA:  2
"Something bad happen with the explicit wait "
"Something really bad happend, with: tipo_viajero span[class="location-review-review-list-parts-TripType__trip_type--3w17i"], could be a change of the HTML DOM "
50
50
50
{'Habitaciones': '50', 'Servicio': '50', 'Calidad del descanso': '50', 'Calidad/precio': '0', 'Ubicación': '0', 'Limpieza': '0'}
"Clicking again show more "
"Something bad happen with the explicit wait "
"Something really bad happend, with: divs_evaluaciones div[class="location-review-review-list-parts-AdditionalRatings__rating--1_G5W location-review-review-list-parts-AdditionalRatings__large--IOg2u"], could be a change of the HTML DOM "
"Clicking again show more "
"Something bad happen with the explicit wait "
"Something really bad happend, with: tipo_viajero span[class="location-review-review-list-parts-TripType__trip_type--3w17i"], could be a change of the HTML DOM "
"Something bad happen with the explicit wait "
"Something real

Page ready...
Datos correctamente guardados
Fishing the selenium web driver
